# Load packages

In [1]:
# skip reinstalling packages we already have
using Pkg

pkgs = [
    "MLJ", "MLJBase", "MLJModels", "MLJEnsembles", "MLJLinearModels",
    "DecisionTree", "MLJDecisionTreeInterface", "NaiveBayes", 
    "MLJNaiveBayesInterface", "EvoTrees", "CategoricalArrays", "Random",
    "LIBSVM", "MLJLIBSVMInterface", "Plots", "MLJModelInterface",
    "CSV", "DataFrames", "UrlDownload", "XGBoost", "NNlib"
]

# Filter out packages already installed
missing_pkgs = filter(pkg -> !(pkg in keys(Pkg.project().dependencies)), pkgs)

if !isempty(missing_pkgs)
    println("Installing missing packages: ", missing_pkgs)
    Pkg.add(missing_pkgs)
else
    println(" All required packages are already installed.")
end


 All required packages are already installed.


In [4]:
using MLJ
using LIBSVM
using NNlib
using Flux
using Flux.Losses
using Statistics

In [ ]:
#Load your library of functions
include("utils.2.2.jl")
# Set a global random seed for reproducibility
using Random
Random.seed!(42)

TaskLocalRNG()

# Load Data

In [1]:
using CSV, DataFrames, Random
using CategoricalArrays

df = CSV.read("./data/updated_pollution_dataset.csv", DataFrame)

# Some log
println("First 5 rows of df:")
show(df[1:5, :], allcols=true)


# Convert last column to categorical (in-place!)
df[!, end] = categorical(df[!, end])

# Extract the integer codes of the categories
targets = Float32.(levelcode.(df[!, end]))

# Use all columns except the last one as inputs
inputs = Matrix{Float32}(df[:, 1:end-1])

println("First 5 inputs::")
for i in 1:5
    println(inputs[i, :])
end

println("\n\nFirst 5 targets:")
println(targets[1:5])

First 5 rows of df:
5×10 DataFrame
 Row │ Temperature  Humidity  PM2.5    PM10     NO2      SO2      CO       Proximity_to_Industrial_Areas  Population_Density  Air Quality 
     │ Float64      Float64   Float64  Float64  Float64  Float64  Float64  Float64                        Int64               String15    
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │        29.8      59.1      5.2     17.9     18.9      9.2     1.72                            6.3                 319  Moderate
   2 │        28.3      75.6      2.3     12.2     30.8      9.7     1.64                            6.0                 611  Moderate
   3 │        23.1      74.7     26.7     33.8     24.4     12.6     1.63                            5.2                 619  Moderate
   4 │        27.1      39.1      6.1      6.3     13.5      5.3     1.15                           11.1                 551  Good
   5 │      

In [7]:
results = Dict()
crossValidationIndices = crossvalidation(targets, 5)

5000-element Vector{Int64}:
 1
 3
 5
 4
 1
 3
 3
 5
 1
 1
 5
 1
 2
 ⋮
 3
 4
 4
 4
 5
 1
 2
 4
 4
 3
 5
 1

In [6]:
function printExperimentResult(model, hyperparams, results)
    (
        (accuracy_mean, accuracy_std),
        (error_rate_mean, error_rate_std),
        (sensitivity_mean, sensitivity_std),
        (specificity_mean, specificity_std),
        (ppv_mean, ppv_std),
        (npv_mean, npv_std),
        (f1_mean, f1_std),
        cm
    ) = results

    println("\n=====================================================")
    println(" Model: $model")
    println(" Hyperparameters: $hyperparams")
    println("=====================================================")

    println(" Accuracy (mean)               : ", round(accuracy_mean, digits=4))
    println(" Accuracy (std)                : ", round(accuracy_std, digits=4))

    println(" Error Rate (mean)             : ", round(error_rate_mean, digits=4))
    println(" Error Rate (std)              : ", round(error_rate_std, digits=4))

    println(" Sensitivity/Recall (mean)     : ", round(sensitivity_mean, digits=4))
    println(" Sensitivity/Recall (std)      : ", round(sensitivity_std,  digits=4))

    println(" Specificity (mean)            : ", round(specificity_mean, digits=4))
    println(" Specificity (std)             : ", round(specificity_std,  digits=4))

    println(" PPV (mean)                    : ", round(ppv_mean,         digits=4))
    println(" PPV (std)                     : ", round(ppv_std,          digits=4))

    println(" NPV (mean)                    : ", round(npv_mean,         digits=4))
    println(" NPV (std)                     : ", round(npv_std,          digits=4))

    println(" F1 Score (mean)               : ", round(f1_mean,          digits=4))
    println(" F1 Score (std)                : ", round(f1_std,           digits=4))

    println("\nConfusion Matrix:")
    display(cm)

    println("=====================================================\n")
end


printExperimentResult (generic function with 1 method)

# Artificial Neural Networks

In [ ]:
############# 1. ARTIFICIAL NEURAL NETWORKS (8+ topologies) #############
default_ann = Dict(      
    "numExecutions" => 5,
    "transferFunctions" => [σ, σ, σ],
    "maxEpochs" => 50,
    "minLoss" => 0.0,
    "learningRate" => 0.01,
    "validationRatio" => 0,
    "maxEpochsVal" => 10
)

ann_search_space = [
    #Dict("topology"=>[8,1]),
    #Dict("topology"=>[10,5,2]),
    #Dict("topology"=>[12,6,3]),
    #Dict("topology"=>[6,3]),
    #Dict("topology"=>[20,10,5]),
    Dict("topology"=>[5,4,3]),
    #Dict("topology"=>[16,8,4]),
    #Dict("topology"=>[30,15,7])
]

1-element Vector{Dict{String, Vector{Int64}}}:
 Dict("topology" => [5, 4, 3])

In [ ]:
########################
# 1. ANN GRID SEARCH
########################
ann_results = []

for hp in ann_search_space
    println("\n=== ANN experiment: topology = $(hp["topology"]) ===")
    full_hp = merge(default_ann, hp)
    res = modelCrossValidation(:ANN, full_hp, (inputs, targets), crossValidationIndices)
    push!(ann_results, (model=:ANN, hyperparams=hp, results=res))
end

results[:ANN] = ann_results


=== ANN experiment: topology = [5, 4, 3] ===


1-element Vector{Any}:
 (model = :ANN, hyperparams = Dict("topology" => [5, 4, 3]), results = ((0.93275994f0, 0.009923107f0), (0.06724f0, 0.009923102f0), (0.93275994f0, 0.009923107f0), (0.97973526f0, 0.0032605396f0), (0.9274502f0, 0.016167527f0), (0.9845723f0, 0.002047198f0), (0.9288909f0, 0.013241017f0), Float32[333.56 0.88 5.1600003 0.4; 0.88 244.92 8.04 6.1600003; 14.079999 8.68 173.84 3.4; 0.4 8.44 10.719999 180.44]))

In [14]:
for entry in results[:ANN]
    printExperimentResult(entry.model, entry.hyperparams, entry.results)
end


 Model: ANN
 Hyperparameters: Dict("topology" => [5, 4, 3])
 Accuracy (mean)               : 0.9328
 Accuracy (std)                : 0.0099
 Error Rate (mean)             : 0.0672
 Error Rate (std)              : 0.0099
 Sensitivity/Recall (mean)     : 0.9328
 Sensitivity/Recall (std)      : 0.0099
 Specificity (mean)            : 0.9797
 Specificity (std)             : 0.0033
 PPV (mean)                    : 0.9275
 PPV (std)                     : 0.0162
 NPV (mean)                    : 0.9846
 NPV (std)                     : 0.002
 F1 Score (mean)               : 0.9289
 F1 Score (std)                : 0.0132

Confusion Matrix:


4×4 Matrix{Float32}:
 333.56    0.88    5.16    0.4
   0.88  244.92    8.04    6.16
  14.08    8.68  173.84    3.4
   0.4     8.44   10.72  180.44

# Support Vector Machines

In [8]:
SVMClassifier = MLJ.@load SVC pkg=LIBSVM verbosity=0

MLJLIBSVMInterface.SVC

In [ ]:
############# 2. SVM (8+ configs: kernels × C) #############
default_svm = Dict(
    "gamma" => 1.0,
    "degree" => 3,
    "coef0" => 0.0
)
svm_search_space = [
    Dict("kernel"=>"linear", "C"=>0.1),
    Dict("kernel"=>"linear", "C"=>1.0),
    Dict("kernel"=>"linear", "C"=>10.0),

    Dict("kernel"=>"rbf", "C"=>1.0),
    Dict("kernel"=>"rbf", "C"=>10.0),

    Dict("kernel"=>"sigmoid", "C"=>1.0),
    Dict("kernel"=>"poly", "C"=>1.0),
    Dict("kernel"=>"poly", "C"=>5.0),
]

8-element Vector{Dict{String, Any}}:
 Dict("C" => 0.1, "kernel" => "linear")
 Dict("C" => 1.0, "kernel" => "linear")
 Dict("C" => 10.0, "kernel" => "linear")
 Dict("C" => 1.0, "kernel" => "rbf", "gamma" => 2.0)
 Dict("C" => 10.0, "kernel" => "rbf", "gamma" => 0.5)
 Dict("C" => 1.0, "kernel" => "sigmoid", "gamma" => 1.0)
 Dict("C" => 1.0, "kernel" => "poly", "gamma" => 1.0, "degree" => 3)
 Dict("C" => 5.0, "kernel" => "poly", "gamma" => 0.5, "degree" => 4)

In [ ]:
########################
# 2. SVM GRID SEARCH
########################
svm_results = []

for hp in svm_search_space
    println("\n=== SVM experiment: kernel=$(hp["kernel"]) C=$(get(hp,"C","-")) ===")
    full_hp = merge(default_svm, hp)
    res = modelCrossValidation(:SVC, full_hp, (inputs, targets), crossValidationIndices)
    push!(svm_results, (model=:SVC, hyperparams=hp, results=res))
end

results[:SVC] = svm_results


=== SVM experiment: kernel=linear C=0.1 ===

=== SVM experiment: kernel=linear C=1.0 ===

=== SVM experiment: kernel=linear C=10.0 ===

=== SVM experiment: kernel=rbf C=1.0 ===

=== SVM experiment: kernel=rbf C=10.0 ===

=== SVM experiment: kernel=sigmoid C=1.0 ===

=== SVM experiment: kernel=poly C=1.0 ===

=== SVM experiment: kernel=poly C=5.0 ===


8-element Vector{Any}:
 (model = :SVC, hyperparams = Dict{String, Any}("C" => 0.1, "kernel" => "linear"), results = ((0.9252f0, 0.0096020885f0), (0.0748f0, 0.009602084f0), (0.9252f0, 0.0096020885f0), (0.97350633f0, 0.0036459977f0), (0.9255888f0, 0.008998138f0), (0.98270977f0, 0.001990787f0), (0.92350817f0, 0.010069727f0), Float32[332.0 3.2 3.4 1.4; 0.2 241.8 9.8 8.2; 18.8 9.0 170.6 1.6; 2.0 6.8 10.4 180.8]))
 (model = :SVC, hyperparams = Dict{String, Any}("C" => 1.0, "kernel" => "linear"), results = ((0.9374f0, 0.00572712f0), (0.0626f0, 0.005727128f0), (0.9374f0, 0.00572712f0), (0.9799453f0, 0.002152039f0), (0.93695223f0, 0.005510538f0), (0.98477143f0, 0.0015064947f0), (0.9366585f0, 0.005720074f0), Float32[333.4 1.0 5.4 0.2; 0.2 244.4 7.6 7.8; 15.6 8.0 173.4 3.0; 1.0 5.6 7.2 186.2]))
 (model = :SVC, hyperparams = Dict{String, Any}("C" => 10.0, "kernel" => "linear"), results = ((0.93999994f0, 0.0067453594f0), (0.060000002f0, 0.006745369f0), (0.93999994f0, 0.0067453594f0), (0.9816389f0, 

In [11]:
for entry in results[:SVC]
    printExperimentResult(entry.model, entry.hyperparams, entry.results)
end


 Model: SVC
 Hyperparameters: Dict{String, Any}("C" => 0.1, "kernel" => "linear")
 Accuracy (mean)               : 0.9252
 Accuracy (std)                : 0.0096
 Error Rate (mean)             : 0.0748
 Error Rate (std)              : 0.0096
 Sensitivity/Recall (mean)     : 0.9252
 Sensitivity/Recall (std)      : 0.0096
 Specificity (mean)            : 0.9735
 Specificity (std)             : 0.0036
 PPV (mean)                    : 0.9256
 PPV (std)                     : 0.009
 NPV (mean)                    : 0.9827
 NPV (std)                     : 0.002
 F1 Score (mean)               : 0.9235
 F1 Score (std)                : 0.0101

Confusion Matrix:


4×4 Matrix{Float32}:
 332.0    3.2    3.4    1.4
   0.2  241.8    9.8    8.2
  18.8    9.0  170.6    1.6
   2.0    6.8   10.4  180.8



 Model: SVC
 Hyperparameters: Dict{String, Any}("C" => 1.0, "kernel" => "linear")
 Accuracy (mean)               : 0.9374
 Accuracy (std)                : 0.0057
 Error Rate (mean)             : 0.0626
 Error Rate (std)              : 0.0057
 Sensitivity/Recall (mean)     : 0.9374
 Sensitivity/Recall (std)      : 0.0057
 Specificity (mean)            : 0.9799
 Specificity (std)             : 0.0022
 PPV (mean)                    : 0.937
 PPV (std)                     : 0.0055
 NPV (mean)                    : 0.9848
 NPV (std)                     : 0.0015
 F1 Score (mean)               : 0.9367
 F1 Score (std)                : 0.0057

Confusion Matrix:


4×4 Matrix{Float32}:
 333.4    1.0    5.4    0.2
   0.2  244.4    7.6    7.8
  15.6    8.0  173.4    3.0
   1.0    5.6    7.2  186.2



 Model: SVC
 Hyperparameters: Dict{String, Any}("C" => 10.0, "kernel" => "linear")
 Accuracy (mean)               : 0.94
 Accuracy (std)                : 0.0067
 Error Rate (mean)             : 0.06
 Error Rate (std)              : 0.0067
 Sensitivity/Recall (mean)     : 0.94
 Sensitivity/Recall (std)      : 0.0067
 Specificity (mean)            : 0.9816
 Specificity (std)             : 0.0018
 PPV (mean)                    : 0.9395
 PPV (std)                     : 0.0067
 NPV (mean)                    : 0.985
 NPV (std)                     : 0.0017
 F1 Score (mean)               : 0.9395
 F1 Score (std)                : 0.0067

Confusion Matrix:


...

4×4 Matrix{Float32}:
 296.4   16.2   3.0   24.4
  27.2  153.4  30.8   48.6
 116.0   12.2  43.6   28.2
  35.2   27.2  15.6  122.0



 Model: SVC
 Hyperparameters: Dict{String, Any}("C" => 1.0, "kernel" => "poly", "gamma" => 1.0, "degree" => 3)
 Accuracy (mean)               : 0.9432
 Accuracy (std)                : 0.0085
 Error Rate (mean)             : 0.0568
 Error Rate (std)              : 0.0085
 Sensitivity/Recall (mean)     : 0.9432
 Sensitivity/Recall (std)      : 0.0085
 Specificity (mean)            : 0.9828
 Specificity (std)             : 0.002
 PPV (mean)                    : 0.9426
 PPV (std)                     : 0.0084
 NPV (mean)                    : 0.9861
 NPV (std)                     : 0.0022
 F1 Score (mean)               : 0.9427
 F1 Score (std)                : 0.0084

Confusion Matrix:


4×4 Matrix{Float32}:
 333.8    0.6    5.4    0.2
   0.2  244.8    8.2    6.8
  13.4    7.2  176.0    3.4
   0.6    4.8    6.0  188.6



 Model: SVC
 Hyperparameters: Dict{String, Any}("C" => 5.0, "kernel" => "poly", "gamma" => 0.5, "degree" => 4)
 Accuracy (mean)               : 0.944
 Accuracy (std)                : 0.0074
 Error Rate (mean)             : 0.056
 Error Rate (std)              : 0.0074
 Sensitivity/Recall (mean)     : 0.944
 Sensitivity/Recall (std)      : 0.0074
 Specificity (mean)            : 0.983
 Specificity (std)             : 0.0017
 PPV (mean)                    : 0.9435
 PPV (std)                     : 0.0073
 NPV (mean)                    : 0.9864
 NPV (std)                     : 0.0019
 F1 Score (mean)               : 0.9435
 F1 Score (std)                : 0.0074

Confusion Matrix:


4×4 Matrix{Float32}:
 333.8    0.8    5.2    0.2
   0.2  245.2    8.0    6.6
  12.8    7.4  176.0    3.8
   0.4    4.8    5.8  189.0

# Decission Trees

In [33]:
DTClassifier = MLJ.@load DecisionTreeClassifier pkg=DecisionTree verbosity=0

MLJDecisionTreeInterface.DecisionTreeClassifier

In [ ]:
############# 3. DECISION TREES (6 depths) #############
default_dt = Dict(
    "rng" => Random.MersenneTwister(1)
)

dt_search_space = [
    Dict("max_depth"=>2),
    Dict("max_depth"=>3),
    Dict("max_depth"=>4),
    Dict("max_depth"=>5),
    Dict("max_depth"=>6),
    Dict("max_depth"=>8)
]

6-element Vector{Dict{String, Int64}}:
 Dict("max_depth" => 2)
 Dict("max_depth" => 3)
 Dict("max_depth" => 4)
 Dict("max_depth" => 5)
 Dict("max_depth" => 6)
 Dict("max_depth" => 8)

In [ ]:
########################
# 3. DECISION TREE GRID SEARCH
########################
dt_results = []

for hp in dt_search_space
    println("\n=== Decision Tree experiment: max_depth=$(hp["max_depth"]) ===")
    full_hp = merge(default_dt, hp) 
    res = modelCrossValidation(:DecisionTreeClassifier, full_hp, (inputs, targets), crossValidationIndices)
    push!(dt_results, (model=:DT, hyperparams=hp, results=res))
end

results[:DT] = dt_results


=== Decision Tree experiment: max_depth=2 ===

=== Decision Tree experiment: max_depth=3 ===

=== Decision Tree experiment: max_depth=4 ===

=== Decision Tree experiment: max_depth=5 ===

=== Decision Tree experiment: max_depth=6 ===

=== Decision Tree experiment: max_depth=8 ===


6-element Vector{Any}:
 (model = :DT, hyperparams = Dict("max_depth" => 2), results = ((0.82600003f0, 0.006403119f0), (0.174f0, 0.006403126f0), (0.82600003f0, 0.006403119f0), (0.94548815f0, 0.0027086502f0), (0.76008487f0, 0.0067073274f0), (0.96535254f0, 0.0016535282f0), (0.7896253f0, 0.006155786f0), Float32[303.2 10.4 1.8 4.6; 30.0 147.0 3.2 19.8; 1.8 10.2 348.0 0.0; 13.6 59.2 19.4 27.8]))
 (model = :DT, hyperparams = Dict("max_depth" => 3), results = ((0.88780004f0, 0.010084653f0), (0.1122f0, 0.010084643f0), (0.88780004f0, 0.010084653f0), (0.96926343f0, 0.0031553009f0), (0.8885791f0, 0.011808219f0), (0.97291136f0, 0.0027606029f0), (0.88561046f0, 0.010569969f0), Float32[303.2 10.4 1.8 4.6; 15.4 156.0 3.2 25.4; 1.8 5.8 345.2 7.2; 5.4 24.0 7.2 83.4]))
 (model = :DT, hyperparams = Dict("max_depth" => 4), results = ((0.9021999f0, 0.01261745f0), (0.0978f0, 0.012617449f0), (0.9021999f0, 0.01261745f0), (0.9731968f0, 0.002997168f0), (0.9028619f0, 0.013072495f0), (0.97457254f0, 0.003307027f0), 

In [36]:
for entry in results[:DT]
    printExperimentResult(entry.model, entry.hyperparams, entry.results)
end


 Model: DT
 Hyperparameters: Dict("max_depth" => 2)
 Accuracy (mean)               : 0.826
 Accuracy (std)                : 0.0064
 Error Rate (mean)             : 0.174
 Error Rate (std)              : 0.0064
 Sensitivity/Recall (mean)     : 0.826
 Sensitivity/Recall (std)      : 0.0064
 Specificity (mean)            : 0.9455
 Specificity (std)             : 0.0027
 PPV (mean)                    : 0.7601
 PPV (std)                     : 0.0067
 NPV (mean)                    : 0.9654
 NPV (std)                     : 0.0017
 F1 Score (mean)               : 0.7896
 F1 Score (std)                : 0.0062

Confusion Matrix:


4×4 Matrix{Float32}:
 303.2   10.4    1.8   4.6
  30.0  147.0    3.2  19.8
   1.8   10.2  348.0   0.0
  13.6   59.2   19.4  27.8



 Model: DT
 Hyperparameters: Dict("max_depth" => 3)
 Accuracy (mean)               : 0.8878
 Accuracy (std)                : 0.0101
 Error Rate (mean)             : 0.1122
 Error Rate (std)              : 0.0101
 Sensitivity/Recall (mean)     : 0.8878
 Sensitivity/Recall (std)      : 0.0101
 Specificity (mean)            : 0.9693
 Specificity (std)             : 0.0032
 PPV (mean)                    : 0.8886
 PPV (std)                     : 0.0118
 NPV (mean)                    : 0.9729
 NPV (std)                     : 0.0028
 F1 Score (mean)               : 0.8856
 F1 Score (std)                : 0.0106

Confusion Matrix:


4×4 Matrix{Float32}:
 303.2   10.4    1.8   4.6
  15.4  156.0    3.2  25.4
   1.8    5.8  345.2   7.2
   5.4   24.0    7.2  83.4



 Model: DT
 Hyperparameters: Dict("max_depth" => 4)
 Accuracy (mean)               : 0.9022
 Accuracy (std)                : 0.0126
 Error Rate (mean)             : 0.0978
 Error Rate (std)              : 0.0126
 Sensitivity/Recall (mean)     : 0.9022
 Sensitivity/Recall (std)      : 0.0126
 Specificity (mean)            : 0.9732
 Specificity (std)             : 0.003
 PPV (mean)                    : 0.9029
 PPV (std)                     : 0.0131
 NPV (mean)                    : 0.9746
 NPV (std)                     : 0.0033
 F1 Score (mean)               : 0.9013
 F1 Score (std)                : 0.0131

Confusion Matrix:


...

4×4 Matrix{Float32}:
 301.4   12.0    1.4   5.2
   9.0  176.2    3.2  11.6
   2.0    5.0  350.0   3.0
   3.4   25.2    7.0  84.4



 Model: DT
 Hyperparameters: Dict("max_depth" => 6)
 Accuracy (mean)               : 0.919
 Accuracy (std)                : 0.0085
 Error Rate (mean)             : 0.081
 Error Rate (std)              : 0.0085
 Sensitivity/Recall (mean)     : 0.919
 Sensitivity/Recall (std)      : 0.0085
 Specificity (mean)            : 0.9775
 Specificity (std)             : 0.0031
 PPV (mean)                    : 0.9208
 PPV (std)                     : 0.0095
 NPV (mean)                    : 0.9791
 NPV (std)                     : 0.0013
 F1 Score (mean)               : 0.9188
 F1 Score (std)                : 0.0089

Confusion Matrix:


4×4 Matrix{Float32}:
 302.8   10.6    1.4   5.2
   8.8  173.4    3.4  14.4
   2.0    4.8  350.6   2.6
   3.0   18.6    6.2  92.2



 Model: DT
 Hyperparameters: Dict("max_depth" => 8)
 Accuracy (mean)               : 0.9244
 Accuracy (std)                : 0.0099
 Error Rate (mean)             : 0.0756
 Error Rate (std)              : 0.0099
 Sensitivity/Recall (mean)     : 0.9244
 Sensitivity/Recall (std)      : 0.0099
 Specificity (mean)            : 0.9784
 Specificity (std)             : 0.0034
 PPV (mean)                    : 0.9234
 PPV (std)                     : 0.0108
 NPV (mean)                    : 0.9817
 NPV (std)                     : 0.0021
 F1 Score (mean)               : 0.9234
 F1 Score (std)                : 0.0105

Confusion Matrix:


4×4 Matrix{Float32}:
 309.8    6.2    2.4   1.6
  11.6  172.2    1.4  14.8
   1.6    5.4  348.2   4.8
   4.2   17.2    4.4  94.2

# K-Nearest Neighbors

In [37]:
kNNClassifier = MLJ.@load KNNClassifier pkg=NearestNeighborModels verbosity=0

NearestNeighborModels.KNNClassifier

In [38]:
############# 4. kNN (6 values) #############
knn_search_space = [
    Dict("K"=>1),
    Dict("K"=>3),
    Dict("K"=>5),
    Dict("K"=>7),
    Dict("K"=>9),
    Dict("K"=>11)
]

6-element Vector{Dict{String, Int64}}:
 Dict("K" => 1)
 Dict("K" => 3)
 Dict("K" => 5)
 Dict("K" => 7)
 Dict("K" => 9)
 Dict("K" => 11)

In [39]:
########################
# 4. KNN GRID SEARCH
########################
knn_results = []

for hp in knn_search_space
    println("\n=== kNN experiment: K=$(hp["K"]) ===")
    res = modelCrossValidation(:KNeighborsClassifier, hp, (inputs, targets), crossValidationIndices)
    push!(knn_results, (model=:KNN, hyperparams=hp, results=res))
end

results[:KNN] = knn_results


=== kNN experiment: K=1 ===

=== kNN experiment: K=3 ===

=== kNN experiment: K=5 ===

=== kNN experiment: K=7 ===

=== kNN experiment: K=9 ===

=== kNN experiment: K=11 ===


6-element Vector{Any}:
 (model = :KNN, hyperparams = Dict("K" => 1), results = ((0.79139996f0, 0.0077006603f0), (0.2086f0, 0.0077006486f0), (0.79139996f0, 0.0077006603f0), (0.9237429f0, 0.0017236611f0), (0.7870192f0, 0.007559483f0), (0.932947f0, 0.0025469856f0), (0.78860193f0, 0.0076150573f0), Float32[257.8 25.2 28.0 9.0; 40.2 130.0 8.4 21.4; 12.8 10.6 331.4 5.2; 13.0 26.4 8.4 72.2]))
 (model = :KNN, hyperparams = Dict("K" => 3), results = ((0.79499996f0, 0.0041833105f0), (0.205f0, 0.004183299f0), (0.79499996f0, 0.0041833105f0), (0.9236349f0, 0.0031811905f0), (0.78815323f0, 0.005101595f0), (0.9371176f0, 0.0007465695f0), (0.79049337f0, 0.004844953f0), Float32[261.4 18.2 32.0 8.4; 43.0 127.8 5.0 24.2; 9.6 8.8 335.4 6.2; 12.2 28.6 8.8 70.4]))
 (model = :KNN, hyperparams = Dict("K" => 5), results = ((0.79919994f0, 0.014652638f0), (0.20079999f0, 0.014652644f0), (0.79919994f0, 0.014652638f0), (0.9219071f0, 0.0052363737f0), (0.7925474f0, 0.015869232f0), (0.93967706f0, 0.0048582363f0), (0.7934

In [40]:
for entry in results[:KNN]
    printExperimentResult(entry.model, entry.hyperparams, entry.results)
end


 Model: KNN
 Hyperparameters: Dict("K" => 1)
 Accuracy (mean)               : 0.7914
 Accuracy (std)                : 0.0077
 Error Rate (mean)             : 0.2086
 Error Rate (std)              : 0.0077
 Sensitivity/Recall (mean)     : 0.7914
 Sensitivity/Recall (std)      : 0.0077
 Specificity (mean)            : 0.9237
 Specificity (std)             : 0.0017
 PPV (mean)                    : 0.787
 PPV (std)                     : 0.0076
 NPV (mean)                    : 0.9329
 NPV (std)                     : 0.0025
 F1 Score (mean)               : 0.7886
 F1 Score (std)                : 0.0076

Confusion Matrix:


4×4 Matrix{Float32}:
 257.8   25.2   28.0   9.0
  40.2  130.0    8.4  21.4
  12.8   10.6  331.4   5.2
  13.0   26.4    8.4  72.2



 Model: KNN
 Hyperparameters: Dict("K" => 3)
 Accuracy (mean)               : 0.795
 Accuracy (std)                : 0.0042
 Error Rate (mean)             : 0.205
 Error Rate (std)              : 0.0042
 Sensitivity/Recall (mean)     : 0.795
 Sensitivity/Recall (std)      : 0.0042
 Specificity (mean)            : 0.9236
 Specificity (std)             : 0.0032
 PPV (mean)                    : 0.7882
 PPV (std)                     : 0.0051
 NPV (mean)                    : 0.9371
 NPV (std)                     : 0.0007
 F1 Score (mean)               : 0.7905
 F1 Score (std)                : 0.0048

Confusion Matrix:


4×4 Matrix{Float32}:
 261.4   18.2   32.0   8.4
  43.0  127.8    5.0  24.2
   9.6    8.8  335.4   6.2
  12.2   28.6    8.8  70.4



 Model: KNN
 Hyperparameters: Dict("K" => 5)
 Accuracy (mean)               : 0.7992
 Accuracy (std)                : 0.0147
 Error Rate (mean)             : 0.2008
 Error Rate (std)              : 0.0147
 Sensitivity/Recall (mean)     : 0.7992
 Sensitivity/Recall (std)      : 0.0147
 Specificity (mean)            : 0.9219
 Specificity (std)             : 0.0052
 PPV (mean)                    : 0.7925
 PPV (std)                     : 0.0159
 NPV (mean)                    : 0.9397
 NPV (std)                     : 0.0049
 F1 Score (mean)               : 0.7934
 F1 Score (std)                : 0.0155

Confusion Matrix:


...

4×4 Matrix{Float32}:
 269.2   14.8   31.0   5.0
  47.4  129.2    4.0  19.4
   9.4   10.8  337.0   2.8
  16.0   29.0    9.4  65.6



 Model: KNN
 Hyperparameters: Dict("K" => 9)
 Accuracy (mean)               : 0.7996
 Accuracy (std)                : 0.0116
 Error Rate (mean)             : 0.2004
 Error Rate (std)              : 0.0116
 Sensitivity/Recall (mean)     : 0.7996
 Sensitivity/Recall (std)      : 0.0116
 Specificity (mean)            : 0.9206
 Specificity (std)             : 0.0044
 PPV (mean)                    : 0.7954
 PPV (std)                     : 0.0137
 NPV (mean)                    : 0.9395
 NPV (std)                     : 0.0036
 F1 Score (mean)               : 0.7938
 F1 Score (std)                : 0.0129

Confusion Matrix:


4×4 Matrix{Float32}:
 269.0   15.0   31.6   4.4
  49.4  128.8    4.4  17.4
  10.0   10.8  336.8   2.4
  16.8   29.6    8.6  65.0



 Model: KNN
 Hyperparameters: Dict("K" => 11)
 Accuracy (mean)               : 0.7916
 Accuracy (std)                : 0.0105
 Error Rate (mean)             : 0.2084
 Error Rate (std)              : 0.0105
 Sensitivity/Recall (mean)     : 0.7916
 Sensitivity/Recall (std)      : 0.0105
 Specificity (mean)            : 0.9162
 Specificity (std)             : 0.004
 PPV (mean)                    : 0.7875
 PPV (std)                     : 0.0129
 NPV (mean)                    : 0.938
 NPV (std)                     : 0.003
 F1 Score (mean)               : 0.7845
 F1 Score (std)                : 0.0119

Confusion Matrix:


4×4 Matrix{Float32}:
 267.2   14.6   34.6   3.6
  52.2  127.0    3.6  17.2
   9.4   11.6  336.6   2.4
  18.2   31.2    9.8  60.8